#7.Code để chạy thuật toán tìm kiếm A*
    Vì hàm heuristic được luyện trên dữ liệu mà ở đó node đích được cố định là Minecraft nên thuật toán chỉ chạy được
    với node đích cố định là Minecraft

In [1]:
import pickle
import time
from heapdict import heapdict


In [2]:
# Load mô hình đã luyện từ file
with open("trained_model.pkl", "rb") as file:
    model = pickle.load(file)

# Load vectorizer từ file
with open("vectorizer.pkl", "rb") as file:
    vectorizer = pickle.load(file)

# Load đồ thị
file_path = r"graph_with_attributes.pkl"
with open(file_path, "rb") as file:
    graph = pickle.load(file)


In [3]:
# Cache để lưu các giá trị heuristic đã tính; hạn chế tính lại nhiều lần
heuristic_cache = {}
def heuristic(source, target):
    # Kiểm tra xem giá trị heuristic đã được tính chưa
    cached_value = heuristic_cache.get(source)
    if cached_value is not None:
        return cached_value
    # Lấy các attributes của node nguồn
    source_attributes = graph.nodes[source]

    # Sắp xếp các attributes thành định dạng đúng
    formatted_attributes = [
        source,
        target,
        source_attributes.get("Word Count"),
        source_attributes.get("Link Count"),
        source_attributes.get("Bag of Words"),
    ]

    # Xử lý các giá trị thiếu trong attributes
    formatted_attributes = ["" if attr is None else attr for attr in formatted_attributes]

    # Tokenize và vectorize formatted attributes dựa trên vectorizer đã load
    formatted_attributes_vectorized = vectorizer.transform([formatted_attributes[4]])

    # Dự đoán path length dựa trên mô hình
    path_length = model.predict(formatted_attributes_vectorized)[0]

    return path_length



In [4]:
def astar(graph, start, goal):
    # Dùng Priority queue để sắp xếp các node; từ đó tìm node có priority thấp nhất
    open_nodes = heapdict()
    open_nodes[start] = 0

    # Từ điển để chứa các path lengths ngắn nhất từ node nguồn
    path_lengths = {node: float("inf") for node in graph.nodes}
    path_lengths[start] = 0

    # Từ điển để chứa node trước trong đường đi
    previous_nodes = {node: None for node in graph.nodes}

    # Đếm số node đã thăm
    visited_nodes = 0

    # Xác định thời gian bắt đầu chạy
    start_time = time.time()

    while open_nodes:
        current_node, _ = open_nodes.popitem()
        visited_nodes += 1

        if current_node == goal:
            # Tạo đường đi từ node nguồn tới đích khi tìm ra đường đi
            path = []
            while current_node is not None:
                path.append(current_node)
                current_node = previous_nodes[current_node]
            path.reverse()
            # Tính thời gian chạy
            runtime = time.time() - start_time
            return path, visited_nodes, runtime, len(path)
        neighbors = set(graph.neighbors(current_node)) #Dùng set để tránh các link bị duplicate
        for neighbor in neighbors:
            path_length = path_lengths[current_node] + 1

            if path_length < path_lengths[neighbor]:
                path_lengths[neighbor] = path_length
                previous_nodes[neighbor] = current_node
                priority = path_length + heuristic(neighbor, goal)
                if neighbor in open_nodes:
                    del open_nodes[neighbor]  # Loại bỏ node neighbor để update priority của nó nếu nó có trong open_nodes
                open_nodes[neighbor] = priority

    # Nếu không có đường đi
    return None, visited_nodes, None, None

In [5]:
# Nhập bài đăng đầu và đích
starting_node = input("Enter the starting node: ")
end_node = input("Enter the goal node: ")

 # Kiểm tra xem chúng có trong đồ thị không
if graph.has_node(starting_node) and graph.has_node(end_node):
    # Tìm đường đi dùng A*
    path, node_count, runtime, path_length = astar(graph, starting_node, end_node)

    if path:
        print("Path:", path)
        print("Number of nodes visited:", node_count)
        print("Path length:", path_length)
        print("Runtime:", runtime)
    else:
        print("No path found.")
else:
    print("Starting node or goal node not found in the graph.")

Enter the starting node: Renaissance architecture
Enter the goal node: Minecraft
Path: ['Renaissance architecture', 'Parthenon', 'Greece', '2011', 'Minecraft']
Number of nodes visited: 5414
Path length: 5
Runtime: 135.88471746444702
